In [10]:
# c3.Experiment.get('Cali_noaa_experiment').remove()

In [ ]:
import requests
import json
import time
import datetime

# %% Helper function to construct request urls
def generate_c3_http_url(c3_type, action):
    """Helper function to construct request urls for C3 Cluster.
    Args:
        c3_type  string of c3 type
        action   string of the action that should be performed on the c3_type
    Returns:
        request_url string for http API calls
    """
    url = "https://devseaweedrc1-seaweed-control.devrc01.c3aids.cloud"
    tag = "devseaweedrc1"
    tenant = "seaweed-control"
    request_url = url + "/" + "api/1" + "/" + tenant + "/" + tag + "/" + c3_type + "?" + "action=" + action
    return request_url

# Step 1: get the auth token

In [25]:
# Note: We can authenticate in two ways:
# 1) Get an oauth token using username & password. After that authenticate with it in the header.
# 2) For every request authenticate via username & password. But this takes significantly longer (+4s per request)
# Not sure how long the oauth token is valid for, I have tested it for 1 day so far.
# For now the workflow below is implemented using Option 2) because it's faster.

In [26]:
# %% Step 1: get the auth token
data = {'grant_type': 'client_credentials', }
response = requests.post(
    'https://devseaweedrc1-seaweed-control.devrc01.c3aids.cloud/oauth/token',
    data=data,
    auth=('mariuswiggert@berkeley.edu', 'dDXAd@ikKM7gihL'),
)
print(response.text)
auth_token = response.json()['access_token']

# Set the header for all future requests
headers = {'content-type': 'application/json', 'accept': 'application/json', 'authorization': 'Bearer ' + auth_token}

{
  "type" : "OAuthAccessTokenResponse",
  "access_token" : "eJwNy8cBA0EIBLCWGMIB5bCE/kuw/5KQCKpA3I44RbR64IwT5HxHxefU94qjMyC+6498YPGI3qml1kTuJsBTViXEI0udmjffX0dh19i+3cnYL7THQrn/f+b43kIkxyY+6VpnscSVpobsDzXWLP4",
  "token_type" : "bearer"
}


# Step 2: Set-up an Experiment, Controller, and Mission (before operation)

## Create Experiment

In [4]:
# Forecast System that we can use:
# - HYCOM Global (daily forecasts, xh resolution, 1/12 deg spatial resolution)
# - Copernicus Global (daily forecasts for 5 days out, xh resolution, 1/12 deg spatial resolution)
# - NOAA West Coast Nowcast System (daily nowcasts for 24h, xh resolution, 10km spatial resolution)

In [8]:
# %% Step 2: Set up an Experiment (collection of missions and controllers, all with same platform specs (thrust))
# Note: It gives a deserialize error, but it creates the object and everything works as expected.
experimentName = 'HalfMoonBayTest_NOAA_April'
max_speed_of_platform_in_meter_per_second = 0.2
forecast_system_to_use = 'noaa' # either of ['HYCOM', 'Copernicus', 'noaa']

# Configs for Experiment
arena_config = {
    'casadi_cache_dict': {
        'deg_around_x_t': 0.5,
        'time_around_x_t': 86400.0},  # This is 24h in seconds!
    'platform_dict': {'battery_cap_in_wh': 400.0,
                      'u_max_in_mps': max_speed_of_platform_in_meter_per_second,
                      'motor_efficiency': 1.0,
                      'solar_panel_size': 1.0,
                      'solar_efficiency': 0.2,
                      'drag_factor': 675.0,
                      'dt_in_s': 600.0},
    'use_geographic_coordinate_system': True,
    'spatial_boundary': None,
    'ocean_dict': {
        'region': 'Region 1', # This is the region of northern California
        'hindcast': {
            "field": "OceanCurrents",
            "source": "opendap",
            "source_settings": {
                "service": "copernicus",
                "currents": "total",
                "USERNAME": "mmariuswiggert",
                "PASSWORD": "tamku3-qetroR-guwneq",
                "DATASET_ID": "cmems_mod_glo_phy_anfc_merged-uv_PT1H-i"}},
        'forecast': {
            'field': 'OceanCurrents',
            'source': 'forecast_files',
            'source_settings': {
                'source': forecast_system_to_use,
                'type': 'forecast'}}},
    'solar_dict': {'hindcast': None, 'forecast': None},
    'seaweed_dict': {'hindcast': None, 'forecast': None}}
# For Navigation Missions (go from A to B) always this Config. Others for Maximizing Growth.
objectiveConfig = {'type': 'nav'}

# send via HTTP request
payload = {
    'name': experimentName,
    'description': 'description of experiment',
    'arenaConfig': json.dumps(arena_config),
    'objectiveConfig': json.dumps(objectiveConfig),
    'timeout_in_sec': 3600 * 24 * 2,  # only relevant for simulation testing of the same scenario
}
# result = requests.post(generate_c3_http_url(c3_type="Experiment", action="createNew"), headers=headers, params=payload)
print(result.json())

NameError: name 'result' is not defined

In [10]:
exp = c3.Experiment.get('HalfMoonBayTest_NOAA_April')

In [12]:
exp.arenaConfig = arena_config

## Create Observer and Controller

In [7]:
# %% Step 3: Add Controller to be used (and Observer, currently none)
# Add observer
payload = {
    'id': experimentName,
    'name': "NoObserver",
    'observerConfig': json.dumps({"observer": None})
}
result = requests.post(generate_c3_http_url(c3_type="Experiment", action="addObserver"), headers=headers,
                       params=payload)
print(result.json())
observer_id = result.json()['id']
# %% add controller
# Config for HJ Controller
ctrl_config = {
    "ctrl_name": "ocean_navigation_simulator.controllers.hj_planners.HJReach2DPlanner.HJReach2DPlanner",
    "replan_on_new_fmrc": True,
    "replan_every_X_seconds": 3600*6,
    "direction": "multi-time-reach-back",
    "deg_around_xt_xT_box": 1.0,  # area over which to run HJ_reachability
    "T_goal_in_seconds": 3600 * 12,
    "grid_res": 0.01,  # Note: this is in deg lat, lon (HYCOM Global is 0.083 and Mexico 0.04)
}
payload = {
    'id': experimentName,
    'name': "HJ_controller_every6h_T_horizon_12h",
    'ctrlConfig': json.dumps(ctrl_config)
}
result = requests.post(generate_c3_http_url(c3_type="Experiment", action="addController"), headers=headers,
                       params=payload)
print(result.json())
controller_id = result.json()['id']

{'name': 'NoObserver', 'id': 'HalfMoonBayTest_NOAA_April_NoObserver', 'meta': {'created': '2023-04-18T23:39:51Z', 'updated': '2023-04-18T23:39:51Z', 'timestamp': '2023-04-18T23:39:51Z'}, 'version': 1}
{'name': 'HJ_controller_every6h_T_horizon_12h', 'id': 'HalfMoonBayTest_NOAA_April_HJ_controller_every6h_T_horizon_12h', 'meta': {'created': '2023-04-18T23:42:04Z', 'updated': '2023-04-18T23:42:04Z', 'timestamp': '2023-04-18T23:42:04Z'}, 'version': 1}


## Add Mission (A -> B)

In [8]:
# %% add a Mission (Start to goal)
print("current UTC datetime is: ", datetime.datetime.now())

current UTC datetime is:  2023-04-18 23:42:12.991473


In [9]:
missionConfig = {
    "target_radius": 0.1,  # in degrees
    "x_0": [{"date_time": "2023-04-19T08:00:00+00:00", "lat": 37.75, "lon": -122.5}],  # the start position and time
    "x_T": {"lat": 37, "lon": -122.4}}  # the target position

payload = {
    'id': experimentName,
    'name': "api_test_mission",
    'missionConfig': json.dumps(missionConfig)
}
result = requests.post(generate_c3_http_url(c3_type="Experiment", action="addMission"), headers=headers, params=payload)
print(result.json())
mission_id = result.json()['id']

{'experiment': {'T_horizon_FC_Error_Calculation_in_h': 120, 'id': 'HalfMoonBayTest_NOAA_April'}, 'status': 'staged_for_feasibility', 'missionConfig': {'target_radius': 0.1, 'x_0': [{'date_time': '2023-04-19T08:00:00+00:00', 'lat': 37.75, 'lon': -122.5}], 'x_T': {'lat': 37, 'lon': -122.4}}, 'id': 'HalfMoonBayTest_NOAA_April_api_test_mission'}


## Create RealOceanSimRun

In [27]:
mission_id = 'HalfMoonBayTest_NOAA_April_api_test_mission'
controller_id = 'HalfMoonBayTest_NOAA_April_HJ_controller_every6h_T_horizon_12h'
observer_id = 'HalfMoonBayTest_NOAA_April_NoObserver'

In [ ]:
# %% create RealOceanSimRun (and plan HJ for the first time) -> hence this takes a while (≈1 minute)
payload = {
    'mission_id': mission_id,
    'controller_id': controller_id,
    'observer_id': observer_id
}
result = requests.post(generate_c3_http_url(c3_type="Experiment", action="createRealOceanRun"), headers=headers,
                       params=payload)
print(result.json())
real_ocean_run_id = result.json()['id']
print("real ocean_run_id: ", real_ocean_run_id)

# Debug what is happening

In [14]:
import numpy as np
import os
import datetime
import xarray as xr

In [15]:
this = c3.RealOceanSimRun.get('HalfMoonBayTest_NOAA_April_api_test_mission_HJ_controller_every6h_T_horizon_12h')
posix_time=None

In [17]:
# def checkForReplanning(this, posix_time=None):
"""Check if there is a new forecast, if yes replan the value function."""
# try:
if posix_time is None:
    current_datetime = datetime.datetime.now(tz=datetime.timezone.utc)
else:
    current_datetime = datetime.datetime.fromtimestamp(posix_time, tz=datetime.timezone.utc)
# Step 0: imports
from ocean_navigation_simulator.problem_factories.Constructor import Constructor
from ocean_navigation_simulator.environment.PlatformState import SpatioTemporalPoint
from ocean_navigation_simulator.environment.Arena import ArenaObservation
from ocean_navigation_simulator.environment.PlatformState import PlatformState
from ocean_navigation_simulator.utils.misc import set_arena_loggers
from ocean_navigation_simulator.data_sources.OceanCurrentField import OceanCurrentField
from ocean_navigation_simulator.environment.ArenaFactory import ArenaFactory
import logging
import os
set_arena_loggers(logging.DEBUG)
# Step 1: Check if replanning is necessary
this = this.get("mission.missionConfig, mission.experiment.timeout_in_sec," +
                "mission.experiment.arenaConfig, mission.experiment.objectiveConfig," +
                "controllerSetting.ctrlConfig, observerSetting.observerConfig, valueFunctionInfo," +
                "recentPlatformState, planningStatus, valueFunction")

# Step 1.2: set-up download for the appropriate FC files
# Set up file paths and download folders
temp_folder = '/tmp/' + this.id + '/'
# create the folder if it doesn't exist yet
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)
extDir = "ocean_sim_run_results/" + this.mission.experiment.id + '/RealOceanSimRuns/' + this.id

# set download directories (ignore set ones in arenaConfig)
arenaConfig = this.mission.experiment.arenaConfig
if arenaConfig['ocean_dict']['forecast'] is None:
    raise ValueError('arenaConfig[ocean_dict][forecast] must be defined for replanning!')

arenaConfig['timeout'] = this.mission.experiment.timeout_in_sec
arenaConfig['ocean_dict']['forecast']['source_settings']['folder'] = '/tmp/forecast_files/' + this.id + '/'
to_download_forecast_files = arenaConfig["ocean_dict"]["forecast"]["source"] == "forecast_files"

# prepping the file download
point_to_check = SpatioTemporalPoint.from_dict(this.mission.missionConfig['x_0'][0])
# retrieve the most recent platform state
state = this.recentPlatformState
print(state)
most_recent_platform_state = PlatformState.from_dict({
    'date_time': str(state.date_time),
    'lon': state.lon,
    'lat': state.lat
})
# time interval to download files from last platform state to current datetime
t_interval = [most_recent_platform_state.date_time - datetime.timedelta(hours=35),
              current_datetime]

# Checks if we shall replan
# Step 1.1: If no valueFunctionInfo exists -> replan
replan = False
if this.valueFunctionInfo is None:
    print("replan because first run!")
    replan = True
elif this.valueFunctionInfo.FC_file_start is None:
    print("replan because errors in first run!")
    replan = True
elif checkIfReplanningBcNewFMRC(this, arenaConfig, t_interval):
    print("replan because new FC file!")
    replan = True
elif this.controllerSetting.ctrlConfig.get('replan_every_X_seconds', False):
    if (current_datetime - this.valueFunctionInfo.date_time).total_seconds() > \
            this.controllerSetting.ctrlConfig['replan_every_X_seconds']:
        print('Replan because of replan_every_X_seconds')
        replan = True
if not replan:
    print('Not Replanning')
    if this.planningStatus != 'no_errors':
        this.planningStatus = 'no_errors'
        this.valueFunctionInfo.info = 'no_errors'
        this.merge()
#     return this.valueFunctionInfo

print('Replanning')
with ArenaFactory.download_files(
        config=arenaConfig, type="forecast", throw_exceptions=False,
        t_interval=t_interval, c3=c3, points=[point_to_check.to_spatial_point()]):

    # Step 1.3.2: instantiate the FC source
    forecast_source_dict = arenaConfig['ocean_dict']['forecast']
    forecast_source_dict["casadi_cache_settings"] = arenaConfig['casadi_cache_dict']
    forecast_source_dict["use_geographic_coordinate_system"] = arenaConfig['use_geographic_coordinate_system']
    forecast_source = OceanCurrentField.instantiate_source_from_dict(forecast_source_dict)

    most_recent_fc_date = forecast_source.check_for_most_recent_fmrc_dataframe(time=current_datetime)
    # Set most_recent_platform_state.date_time to forecast start because we want HJ to plan back as far as it can
    if most_recent_platform_state.date_time < most_recent_fc_date:
        most_recent_platform_state.date_time = forecast_source.check_for_most_recent_fmrc_dataframe(
            time=current_datetime)

    # Step 0: Create Constructor to instantiate problem and controller
    constructor = Constructor(
        arena_conf=arenaConfig,
        mission_conf=this.mission.missionConfig,
        objective_conf=this.mission.experiment.objectiveConfig,
        ctrl_conf=this.controllerSetting.ctrlConfig,
        observer_conf=this.observerSetting.observerConfig,
        create_arena=False
    )

    # Step 1: Trigger the planning
    _ = constructor.controller.get_action(
        observation=ArenaObservation(
            platform_state=most_recent_platform_state,
            true_current_at_state=None,
            forecast_data_source=forecast_source,
        ))

    # Step 4: create a dataset for the HJ Value Function
    hj_values = constructor.controller.all_values
    posix_times = constructor.controller.reach_times + constructor.controller.current_data_t_0
    x_grid = constructor.controller.grid.coordinate_vectors[0]
    y_grid = constructor.controller.grid.coordinate_vectors[1]

    data = xr.Dataset(
        {"HJValueFunc": (("time", "x", "y"), hj_values)},
        coords={"x": x_grid, "y": y_grid, "time": posix_times})

    # Step 4.2 Save to file
    temp_folder = '/tmp/' + this.id + '/'

    nc_file_name = "HJ_func_" + current_datetime.strftime("%m_%d_%Y_%H_%M") + ".nc"
    data.to_netcdf(temp_folder + nc_file_name)

    # Step 4.3 upload file to Azure and delete local file
    c3.Client.uploadLocalClientFiles(temp_folder + nc_file_name, extDir + '/hj_val_funcs',
                                     {"peekForMetadata": True})
    c3_file_obj = c3.File(**{'url': extDir + '/hj_val_funcs/' + nc_file_name}).readMetadata()
    os.remove(temp_folder + nc_file_name)

    # Update RealOceanSimRun
    this = this.get('planningStatus, valueFunctionInfo, valueFunction')
    this.valueFunction = c3_file_obj
    this.valueFunctionInfo = c3.PlanningMetaInfo(**{
        'date_time': current_datetime,
        'FC_file_start': most_recent_fc_date,
        'info': 'no_errors'
    })
    this.planningStatus = 'no_errors'
    this.merge()

# except Exception as e:
#     # if we crash we should upload that to the table for debugging
#     print("Error Message: ", e)
#     this = this.get('oceanSimResult, planningStatus')
#     this.planningStatus = 'error_in_replanning'
#     if this.oceanSimResult is None:
#         this.oceanSimResult = c3.OceanSimResult(**{'error_message': e})
#     else:
#         this.oceanSimResult.error_message = e
#     if this.valueFunctionInfo:
#         this.valueFunctionInfo.info = e
#     else:
#         this.valueFunctionInfo = c3.PlanningMetaInfo(**{'info': e})
#     this.merge()

# return this.valueFunctionInfo

replan because errors in first run!
Replanning


Only 52/97 files in the database for noaa, forecast, Region 1 and t_0=2023-01-12 19:52:40+00:00 and t_T=2023-04-19 00:40:22.615448+00:00: 
- nos.wcofs.regulargrid.f003-f072.20230226.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230227.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230228.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230301.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230302.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230303.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230304.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230305.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230306.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230307.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230308.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230309.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230310.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230311.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230312.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230313.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230314.t0

KeyboardInterrupt: 

# Step 3: Called in operation to get optimal thrust Vector

In [11]:
# %% The platform calls this to get the optimal thurst vector at each point in time
# The server will log both the lat, lon, time of the input and the thrust it computed
# Takes ≈2-3s (first call can take up to 60s because it has to install the python runtime in a VM)
start = time.time()
payload = {
    'id': real_ocean_run_id,
    "lat": 37.1,
    "lon": -130.0,
    "posix_time": 1673679160.0
}
result = requests.post(generate_c3_http_url(c3_type="RealOceanSimRun", action="getThrustVector"), headers=headers,
                       params=payload)
print("took seconds: ", time.time() - start)
print(result.json())

took seconds:  28.320302724838257
{'magnitude': 1.0, 'direction': -0.013157135472755937, 'status': 'Error with HJValueFunction. Default to Naive to Target.'}


# Step 4: Set Status to 'inactive' when Mission is finished

In [53]:
# The RealOceanSimRun object has a status field, as long as this is set to 'active' a job runs 
# to check regularly if a new forecast is available and if it is, the Value Function for control is recomputed.
# This works for as many platforms as we want. When a mission (start to goal) is finished we should set
# the status to inactive to not waste compute.

In [54]:
payload = {
    'ID': real_ocean_run_id,
    "new_status": 'inactive'
}
result = requests.post(generate_c3_http_url(c3_type="RealOceanSimRun", action="updateStatus"), headers=headers,
                       params=payload)
print(result.json())

True
